In [1]:
import pandas as pd
import sasoptpy as so
import random
import string
import os
from subprocess import Popen
pd.options.display.max_colwidth = 200

In [2]:
data = pd.read_csv("data/fplreview.csv", index_col=0)
data['id'] = data.index + 1
data['price'] = data['Value'] / 10
player_dict = data.set_index('id', drop=True).to_dict(orient='index')

In [3]:
data.head()

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
0,47.0,0.174,4,0.274,7,0.280,6,0.286,8,0.358,...,9,0.617,15,0.585,15,G,Leno,Arsenal,1,4.7
1,40.0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,G,Rúnarsson,Arsenal,2,4.0
2,63.0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,M,Willian,Arsenal,3,6.3
3,99.0,5.246,81,4.455,79,5.801,76,3.399,75,5.652,...,72,4.427,69,4.382,70,F,Aubameyang,Arsenal,4,9.9
4,43.0,0.206,5,0.247,7,0.481,10,0.265,10,0.666,...,18,0.730,18,0.631,17,D,Cédric,Arsenal,5,4.3


In [4]:
# Find ID of White
data[data['Name'] == 'White'] # 67

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
66,44.0,3.382,88,2.478,86,3.797,83,1.393,83,3.505,...,75,2.637,78,2.174,73,D,White,Arsenal,67,4.4
598,45.0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,M,White,Newcastle,599,4.5


In [5]:
# Find ID of Livramento
data[data['Name'] == 'Livramento'] # 491

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
490,43.0,3.51,86,3.133,84,3.018,81,3.179,80,1.145,...,77,2.456,74,2.869,73,D,Livramento,Southampton,491,4.3


In [6]:
def get_random_id(n):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(n))

def read_solution(m, sol_file):
    with open(sol_file, 'r') as f:
        for v in m.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            v = m.get_variable(words[1])
            v.set_value(float(words[2]))


def get_solution_info(player_ids, m, gws, xp):
    squad = m.get_variable('squad')
    lineup = m.get_variable('lineup')
    squad_players = []
    lineup_selection = []
    for p in player_ids:
        if squad[p].get_value() > 0.5:
            squad_players.append(p)
    for g in gws:
        for p in squad_players:
            if lineup[p,g].get_value() > 0.5:
                lineup_selection.append({'player': p, 'gw': g, 'xp': round(xp[g][p], 3) })
    return {'squad': squad_players, 'lineup_selection': lineup_selection}


def find_best_rotation(players, N=2, K=1, max_iter=1, first_gw=1, last_gw=38, exclude=[], rules=[]):
    problem_name = get_random_id(10)
    m = so.Model(name=f'pick_{K}_choose_{N}_{problem_name}')
    gameweeks = list(range(first_gw, last_gw+1))
    player_ids = players['id'].tolist()
    squad = m.add_variables(player_ids, vartype=so.binary, name='squad')
    lineup = m.add_variables(player_ids, gameweeks, vartype=so.binary, name='lineup')

    xp = {}
    indexed_players = players.set_index('id')
    for g in gameweeks:
        xp[g] = indexed_players["{}_Pts".format(g)]

    if len(exclude) > 0:
        m.add_constraints((squad[p] == 0 for p in exclude), name='ban_player')

    m.add_constraint(so.expr_sum(squad[p] for p in player_ids) == N, name='select_N')
    m.add_constraints((so.expr_sum(lineup[p, g] for p in player_ids) == K for g in gameweeks), name='pick_K')
    m.add_constraints((lineup[p,g] <= squad[p] for p in player_ids for g in gameweeks), name='valid_picks_only')
    for rule_no, rule in enumerate(rules):
        if rule['bound'] == 'lower':
            m.add_constraint(so.expr_sum(squad[p] for p in rule['ids']) >= rule['value'], name=f'rule_{rule_no}')
        elif rule['bound'] == 'upper':
            m.add_constraint(so.expr_sum(squad[p] for p in rule['ids']) <= rule['value'], name=f'rule_{rule_no}')
        elif rule['bound'] == 'use':
            m.add_constraints((squad[p] * rule['value'] <= so.expr_sum(lineup[p,g] for g in gameweeks) for p in player_ids), name=f'min_use_{p}')

    m.set_objective(-so.expr_sum(xp[g][p] * lineup[p,g] for p in player_ids for g in gameweeks), sense='N', name='total_xp')

    solutions = []

    m.export_mps(f"tmp/{problem_name}.mps")
    command = f"cbc tmp/{problem_name}.mps solve solu tmp/{problem_name}.sol"
    Popen(command).wait()
    read_solution(m, f'tmp/{problem_name}.sol')
    sol = get_solution_info(player_ids, m, gameweeks, xp)

    solutions.append(sol)

    os.unlink(f'tmp/{problem_name}.mps')
    os.unlink(f'tmp/{problem_name}.sol')

    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(squad[p] for p in sol['squad']) <= N-1, name=f'cutoff_{it}')
        m.export_mps(f"tmp/{problem_name}.mps")
        Popen(command).wait()
        read_solution(m, f'tmp/{problem_name}.sol')
        sol = get_solution_info(player_ids, m, gameweeks, xp)
        solutions.append(sol)

        os.unlink(f'tmp/{problem_name}.mps')
        os.unlink(f'tmp/{problem_name}.sol')

    return solutions

In [7]:
# Rule: Have at least 1 of Livramento or White
rules = [
    {'bound': 'lower', 'value': 1, 'ids': [67, 491]}
    ]
# Get only defenders under 4.5 M
budget_defenders = data[(data['Pos'] == 'D') & (data['price'] <= 4.5)]
solutions = find_best_rotation(players=budget_defenders, N=2, K=1, max_iter=30, first_gw=9, last_gw=16, exclude=[], rules=rules)

NOTE: Initialized model pick_1_choose_2_sZRuSoIue3.


In [8]:
summary = []
for sol_no, sol in enumerate(solutions):
    print(f"Solution {sol_no+1}")
    selected_names = []
    details = []
    cost = 0
    for p in sol['squad']:
        name = player_dict[p]['Name']
        games = [str(sel['gw']) for sel in sol['lineup_selection'] if sel['player'] == p]
        selected_names.append(f"{name} ({len(games)})")
        details.append(f"{name}: [{','.join(games)}]")
        cost += player_dict[p]['price']
    selected_names = ", ".join(selected_names)
    details = ", ".join(details)
    print(f"Players: {selected_names}")
    total = 0
    for sel in sol['lineup_selection']:
        print(f"GW{sel['gw']:<2d} {player_dict[sel['player']]['Name']:12s} xP: {sel['xp']}")
        total += sel['xp']
    print(f"Total xP: {total:.2f}")
    summary.append({'name': selected_names, 'cost': cost, 'total': total, 'xp_per_cost': total/cost, 'details': details})

Solution 1
Players: Henry (4), Livramento (4)
GW9  Livramento   xP: 3.51
GW10 Henry        xP: 3.236
GW11 Henry        xP: 4.087
GW12 Livramento   xP: 3.179
GW13 Henry        xP: 3.248
GW14 Livramento   xP: 2.676
GW15 Livramento   xP: 2.861
GW16 Henry        xP: 3.776
Total xP: 26.57
Solution 2
Players: White (3), Henry (5)
GW9  White        xP: 3.382
GW10 Henry        xP: 3.236
GW11 Henry        xP: 4.087
GW12 Henry        xP: 2.971
GW13 White        xP: 3.505
GW14 Henry        xP: 2.268
GW15 White        xP: 2.491
GW16 Henry        xP: 3.776
Total xP: 25.72
Solution 3
Players: White (3), Livramento (5)
GW9  Livramento   xP: 3.51
GW10 Livramento   xP: 3.133
GW11 White        xP: 3.797
GW12 Livramento   xP: 3.179
GW13 White        xP: 3.505
GW14 Livramento   xP: 2.676
GW15 Livramento   xP: 2.861
GW16 White        xP: 2.928
Total xP: 25.59
Solution 4
Players: White (3), Salisu (5)
GW9  Salisu       xP: 3.59
GW10 Salisu       xP: 3.198
GW11 White        xP: 3.797
GW12 Salisu       xP: 3.

In [9]:
print("Pick 2 Use 1 Optimal Pairs with at least one of White/Livramento")
pd.DataFrame(summary)

Pick 2 Use 1 Optimal Pairs with at least one of White/Livramento


,name,cost,total,xp_per_cost,details
0,"Henry (4), Livramento (4)",8.8,26.573,3.019659,"Henry: [10,11,13,16], Livramento: [9,12,14,15]"
1,"White (3), Henry (5)",8.9,25.716,2.889438,"White: [9,13,15], Henry: [10,11,12,14,16]"
2,"White (3), Livramento (5)",8.7,25.589,2.941264,"White: [11,13,16], Livramento: [9,10,12,14,15]"
3,"White (3), Salisu (5)",8.9,25.496,2.864719,"White: [11,13,16], Salisu: [9,10,12,14,15]"
4,"White (5), Coady (3)",8.9,25.172,2.828315,"White: [9,11,13,15,16], Coady: [10,12,14]"
5,"Ajer (3), Livramento (5)",8.8,24.882,2.827500,"Ajer: [11,13,16], Livramento: [9,10,12,14,15]"
6,"White (5), Lowton (3)",8.8,24.383,2.770795,"White: [9,11,13,15,16], Lowton: [10,12,14]"
7,"White (5), Lascelles (3)",8.8,24.320,2.763636,"White: [9,10,11,13,16], Lascelles: [12,14,15]"
8,"Duffy (3), Livramento (5)",8.7,24.295,2.792529,"Duffy: [11,13,16], Livramento: [9,10,12,14,15]"
9,"White (5), Taylor (3)",8.8,24.264,2.757273,"White: [9,11,13,15,16], Taylor: [10,12,14]"


In [10]:
# Rule: Have BOTH Livramento and White, and another defender
rules = [
    {'bound': 'lower', 'value': 2, 'ids': [67, 491]},
    {'bound': 'use', 'value': 1} # Each picked player should play at least this many GWs
    ]
# Get only defenders under 4.5 M
budget_defenders = data[(data['Pos'] == 'D') & (data['price'] <= 4.5)]
solutions = find_best_rotation(players=budget_defenders, N=3, K=1, max_iter=10, first_gw=9, last_gw=16, exclude=[], rules=rules)

NOTE: Initialized model pick_1_choose_3_0r1wdTWCoE.


In [11]:
summary = []
for sol_no, sol in enumerate(solutions):
    print(f"Solution {sol_no+1}")
    selected_names = []
    details = []
    cost = 0
    for p in sol['squad']:
        name = player_dict[p]['Name']
        games = [str(sel['gw']) for sel in sol['lineup_selection'] if sel['player'] == p]
        selected_names.append(f"{name} ({len(games)})")
        details.append(f"{name}: [{','.join(games)}]")
        cost += player_dict[p]['price']
    selected_names = ", ".join(selected_names)
    details = ", ".join(details)
    print(f"Players: {selected_names}")
    total = 0
    for sel in sol['lineup_selection']:
        print(f"GW{sel['gw']:<2d} {player_dict[sel['player']]['Name']:12s} xP: {sel['xp']}")
        total += sel['xp']
    print(f"Total xP: {total:.2f}")
    summary.append({'name': selected_names, 'cost': cost, 'total': total, 'xp_per_cost': total/cost, 'details': details})

Solution 1
Players: White (1), Henry (3), Livramento (4)
GW9  Livramento   xP: 3.51
GW10 Henry        xP: 3.236
GW11 Henry        xP: 4.087
GW12 Livramento   xP: 3.179
GW13 White        xP: 3.505
GW14 Livramento   xP: 2.676
GW15 Livramento   xP: 2.861
GW16 Henry        xP: 3.776
Total xP: 26.83
Solution 2
Players: White (3), Coady (2), Livramento (3)
GW9  Livramento   xP: 3.51
GW10 Coady        xP: 3.156
GW11 White        xP: 3.797
GW12 Livramento   xP: 3.179
GW13 White        xP: 3.505
GW14 Coady        xP: 3.308
GW15 Livramento   xP: 2.861
GW16 White        xP: 2.928
Total xP: 26.24
Solution 3
Players: White (3), Lascelles (1), Livramento (4)
GW9  Livramento   xP: 3.51
GW10 Livramento   xP: 3.133
GW11 White        xP: 3.797
GW12 Livramento   xP: 3.179
GW13 White        xP: 3.505
GW14 Lascelles    xP: 3.127
GW15 Livramento   xP: 2.861
GW16 White        xP: 2.928
Total xP: 26.04
Solution 4
Players: White (3), Kilman (1), Livramento (4)
GW9  Livramento   xP: 3.51
GW10 Livramento   xP: 3

In [12]:
print("Pick 3 Use 1 Optimal Picks with White & Livramento")
pd.DataFrame(summary)

Pick 3 Use 1 Optimal Picks with White & Livramento


,name,cost,total,xp_per_cost,details
0,"White (1), Henry (3), Livramento (4)",13.2,26.830,2.032576,"White: [13], Henry: [10,11,16], Livramento: [9,12,14,15]"
1,"White (3), Coady (2), Livramento (3)",13.2,26.244,1.988182,"White: [11,13,16], Coady: [10,14], Livramento: [9,12,15]"
2,"White (3), Lascelles (1), Livramento (4)",13.1,26.040,1.987786,"White: [11,13,16], Lascelles: [14], Livramento: [9,10,12,15]"
3,"White (3), Kilman (1), Livramento (4)",13.2,25.978,1.968030,"White: [11,13,16], Kilman: [14], Livramento: [9,10,12,15]"
4,"White (3), Salisu (3), Livramento (2)",13.2,25.783,1.953258,"White: [11,13,16], Salisu: [9,10,12], Livramento: [14,15]"
5,"White (2), Ajer (1), Livramento (5)",13.2,25.752,1.950909,"White: [11,13], Ajer: [16], Livramento: [9,10,12,14,15]"
6,"White (3), Cooper (1), Livramento (4)",13.2,25.703,1.947197,"White: [11,13,16], Cooper: [14], Livramento: [9,10,12,15]"
7,"White (3), Mitchell (1), Livramento (4)",13.2,25.522,1.933485,"White: [11,13,16], Mitchell: [9], Livramento: [10,12,14,15]"
8,"White (3), Lowton (1), Livramento (4)",13.1,25.492,1.945954,"White: [11,13,16], Lowton: [12], Livramento: [9,10,14,15]"
9,"White (3), Taylor (1), Livramento (4)",13.1,25.482,1.945191,"White: [11,13,16], Taylor: [12], Livramento: [9,10,14,15]"


In [13]:
print(":)")

:)
